In [1]:
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

from langgraph.graph import START, StateGraph, MessagesState
from langgraph.prebuilt import tools_condition, ToolNode

In [2]:
import os

from dotenv import load_dotenv
load_dotenv()
env = os.environ.copy()

from rich.console import Console
console = Console(width=96)

pubmed_server = "node_modules/@cyanheads/pubmed-mcp-server/dist/index.js"

from system_prompts import PUBMED_AGENT_PROMPT

In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient
client = MultiServerMCPClient(
    {
        "pubmed": {
            "command": "node",
            "args": [pubmed_server],
            "transport": "stdio",
            "env": env
        }
    }
)

In [7]:
tools = await client.get_tools()

In [9]:
for tool in tools:
    print(tool.name)

pubmed_article_connections
pubmed_fetch_contents
pubmed_generate_chart
pubmed_research_agent
pubmed_search_articles


In [10]:
tools

[StructuredTool(name='pubmed_article_connections', description='Finds articles related to a source PubMed ID (PMID) or retrieves formatted citations for it. Supports finding similar articles, articles that cite the source, articles referenced by the source (via NCBI ELink), or fetching data to generate citations in various styles (RIS, BibTeX, APA, MLA via NCBI EFetch and server-side formatting). Returns a JSON object detailing the connections or formatted citations.', args_schema={'type': 'object', 'properties': {'sourcePmid': {'type': 'string', 'pattern': '^\\d+$', 'description': 'The PubMed Unique Identifier (PMID) of the source article for which to find connections or format citations. This PMID must be a valid number string.'}, 'relationshipType': {'type': 'string', 'enum': ['pubmed_similar_articles', 'pubmed_citedin', 'pubmed_references', 'citation_formats'], 'default': 'pubmed_similar_articles', 'description': "Specifies the type of connection or action: 'pubmed_similar_articles

In [11]:
# Find the tool by name
fetch_tool = tools[1]

# Manually invoke it to see the structure for a specific PMID
result = await fetch_tool.ainvoke({
    "pmids": ["31772153"],
    "detailLevel": "full_xml"
})

print(result)


[{'type': 'text', 'text': '{"requestedPmids":["31772153"],"articles":[{"pmid":"31772153","fullXmlContent":{"MedlineCitation":{"PMID":{"#text":31772153,"@_Version":"1"},"DateCompleted":{"Year":2020,"Month":8,"Day":31},"DateRevised":{"Year":2024,"Month":4,"Day":22},"Article":{"Journal":{"ISSN":{"#text":"2041-4889","@_IssnType":"Electronic"},"JournalIssue":{"Volume":10,"Issue":12,"PubDate":{"Year":2019,"Month":"Nov","Day":26},"@_CitedMedium":"Internet"},"Title":"Cell death & disease","ISOAbbreviation":"Cell Death Dis"},"ArticleTitle":"The fungal metabolite chaetocin is a sensitizer for pro-apoptotic therapies in glioblastoma.","Pagination":{"StartPage":894,"MedlinePgn":894},"ELocationID":[{"#text":894,"@_EIdType":"pii","@_ValidYN":"Y"},{"#text":"10.1038/s41419-019-2107-y","@_EIdType":"doi","@_ValidYN":"Y"}],"Abstract":{"AbstractText":"Glioblastoma Multiforme (GBM) is the most common and aggressive primary brain tumor. Despite recent developments in surgery, chemo- and radio-therapy, a cur

In [26]:
from pprint import pprint as pp
import json

# result[0] is the object you provided
# result[0].text contains the JSON string
pubmed_data = json.loads(result[0]['text'])

# The actual articles are in the "articles" list
articles = pubmed_data.get("articles", [])
# Print the first article found
if articles:
    console.print(json.dumps(articles[0], indent=2))


{
  "pmid": "31772153",
  "fullXmlContent": {
    "MedlineCitation": {
      "PMID": {
        "#text": 31772153,
        "@_Version": "1"
      },
      "DateCompleted": {
        "Year": 2020,
        "Month": 8,
        "Day": 31
      },
      "DateRevised": {
        "Year": 2024,
        "Month": 4,
        "Day": 22
      },
      "Article": {
        "Journal": {
          "ISSN": {
            "#text": "2041-4889",
            "@_IssnType": "Electronic"
          },
          "JournalIssue": {
            "Volume": 10,
            "Issue": 12,
            "PubDate": {
              "Year": 2019,
              "Month": "Nov",
              "Day": 26
            },
            "@_CitedMedium": "Internet"
          },
          "Title": "Cell death & disease",
          "ISOAbbreviation": "Cell Death Dis"
        },
        "ArticleTitle": "The fungal metabolite chaetocin is a sensitizer for pro-apoptotic 
therapies in glioblastoma.",
        "Pagination": {
          "StartPage": 894,
          "MedlinePgn": 894
        },
        "ELocationID": [
          {
            "#text": 894,
            "@_EIdType": "pii",
            "@_ValidYN": "Y"
          },
          {
            "#text": "10.1038/s41419-019-2107-y",
            "@_EIdType": "doi",
            "@_ValidYN": "Y"
          }
        ],
        "Abstract": {
          "AbstractText": "Glioblastoma Multiforme (GBM) is the most common and aggressive 
primary brain tumor. Despite recent developments in surgery, chemo- and radio-therapy, a 
currently poor prognosis of GBM patients highlights an urgent need for novel treatment 
strategies. TRAIL (TNF Related Apoptosis Inducing Ligand) is a potent anti-cancer agent that can
induce apoptosis selectively in cancer cells. GBM cells frequently develop resistance to TRAIL 
which renders clinical application of TRAIL therapeutics inefficient. In this study, we 
undertook a chemical screening approach using a library of epigenetic modifier drugs to identify
compounds that could augment TRAIL response. We identified the fungal metabolite chaetocin, an 
inhibitor of histone methyl transferase SUV39H1, as a novel TRAIL sensitizer. Combining low 
subtoxic doses of chaetocin and TRAIL resulted in very potent and rapid apoptosis of GBM cells. 
Chaetocin also effectively sensitized GBM cells to further pro-apoptotic agents, such as FasL 
and BH3 mimetics. Chaetocin mediated apoptosis sensitization was achieved through ROS generation
and consequent DNA damage induction that involved P53 activity. Chaetocin induced transcriptomic
changes showed induction of antioxidant defense mechanisms and DNA damage response pathways. 
Heme Oxygenase 1 (HMOX1) was among the top upregulated genes, whose induction was ROS-dependent 
and HMOX1 depletion enhanced chaetocin mediated TRAIL sensitization. Finally, chaetocin and 
TRAIL combination treatment revealed efficacy in vivo. Taken together, our results provide a 
novel role for chaetocin as an apoptosis priming agent and its combination with pro-apoptotic 
therapies might offer new therapeutic approaches for GBMs."
        },
        "AuthorList": {
          "Author": [
            {
              "LastName": "Ozyerli-Goknar",
              "ForeName": "Ezgi",
              "Initials": "E",
              "AffiliationInfo": {
                "Affiliation": "Brain Cancer Research and Therapy Laboratory, Ko&#xe7; 
University School of Medicine, 34450, Istanbul, Turkey."
              },
              "@_ValidYN": "Y"
            },
            {
              "LastName": "Sur-Erdem",
              "ForeName": "Ilknur",
              "Initials": "I",
              "AffiliationInfo": {
                "Affiliation": "Brain Cancer Research and Therapy Laboratory, Ko&#xe7; 
University School of Medicine, 34450, Istanbul, Turkey."
              },
              "@_ValidYN": "Y"
            },
            {
              "LastName": "Seker",
              "ForeName": "Fidan",
              "Initial

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition

from system_prompts import (
    QUERY_PLANNER_AGENT,
    PUBMED_SEARCH_AGENT,
    PUBMED_FETCH_AGENT,
    METHODS_EXTRACTION_AGENT,
    METHODS_TO_JSON_AGENT,
    PMID_METHODS_FINAL_ASSEMBLY_AGENT,
)

# ---------------------------------------------------------------------
# LLM setup
# ---------------------------------------------------------------------

llm = ChatOpenAI(model="gpt-5-nano")
llm_with_tools = llm.bind_tools(tools)

# ---------------------------------------------------------------------
# Node helpers
# ---------------------------------------------------------------------

def llm_node(system_prompt: str, use_tools: bool = False):
    """
    Factory for simple LLM nodes with fixed system prompts.
    """
    sys_msg = SystemMessage(content=system_prompt)

    def _node(state: MessagesState):
        model = llm_with_tools if use_tools else llm
        response = model.invoke([sys_msg] + state["messages"])
        return {"messages": [response]}

    return _node

# ---------------------------------------------------------------------
# Nodes
# ---------------------------------------------------------------------

query_planner = llm_node(
    system_prompt=QUERY_PLANNER_AGENT,
    use_tools=False,
)

pubmed_search = llm_node(
    system_prompt=PUBMED_SEARCH_AGENT,
    use_tools=True,
)

pubmed_fetch = llm_node(
    system_prompt=PUBMED_FETCH_AGENT,
    use_tools=True,
)

methods_extraction = llm_node(
    system_prompt=METHODS_EXTRACTION_AGENT,
    use_tools=False,
)

methods_to_json = llm_node(
    system_prompt=METHODS_TO_JSON_AGENT,
    use_tools=False,
)

final_assembly = llm_node(
    system_prompt=PMID_METHODS_FINAL_ASSEMBLY_AGENT,
    use_tools=False,
)

# Tool execution node (shared)
tools_node = ToolNode(tools)

# ---------------------------------------------------------------------
# Build graph
# ---------------------------------------------------------------------

builder = StateGraph(MessagesState)

builder.add_node("query_planner", query_planner)
builder.add_node("pubmed_search", pubmed_search)
builder.add_node("pubmed_fetch", pubmed_fetch)
builder.add_node("methods_extraction", methods_extraction)
builder.add_node("methods_to_json", methods_to_json)
builder.add_node("final_assembly", final_assembly)
builder.add_node("tools", tools_node)

# ---------------------------------------------------------------------
# Edges
# ---------------------------------------------------------------------

builder.add_edge(START, "query_planner")
builder.add_edge("query_planner", "pubmed_search")

# Search → tool execution if needed
builder.add_conditional_edges(
    "pubmed_search",
    tools_condition,
)
builder.add_edge("tools", "pubmed_search")

# Fetch → tool execution if needed
builder.add_edge("pubmed_search", "pubmed_fetch")
builder.add_conditional_edges(
    "pubmed_fetch",
    tools_condition,
)
builder.add_edge("tools", "pubmed_fetch")

# Downstream processing (no tools)
builder.add_edge("pubmed_fetch", "methods_extraction")
builder.add_edge("methods_extraction", "methods_to_json")
builder.add_edge("methods_to_json", "final_assembly")
builder.add_edge("final_assembly", END)

# ---------------------------------------------------------------------
# Compile
# ---------------------------------------------------------------------

graph = builder.compile()

